# Automated Data Transform

In this notebook I have created a set of functions to transform data autonomously. The details are below.

A dataflow is a file that is provided by a gas & electricity company.
The D10 dataflow is what they receive every time a customer
sends them a meter reading. They can receive up to 10,000 of these
a day, so we must automate the processing of the data.

Your task is to transform the dataflow from its raw format into
the specified format. The expected output can be found in 
"ExpectedOut.csv". 


# Instructions:



Column Descriptions
*******************

Row Header:
    - column1: Supplier Code (XZ1)
    - column2: Name of the dataflow (DataflowD10)
    - column3: Direction of dataflow
    - column4: Industry code
    - column5: Date and Time file was generated.

Row 026:
    - column1 = Unique ID
    - column2 = Validation status (U = unvalidated, V = validated)
Row 028:
    - column1 = Serial Number
    - column2 = Reading type (C = Customer, A = Agent)
Row 030: 
    - column1 = Meter Register (1, 2, 3 or more - Most customers have 1, 
                                but some customers may have 1, 2 or more!
    - column2 = Reading Date & Time
    - column3 = Meter Reading



Explanation
*************

- Each time there is a row 026, it is a new customer. 
- Each customer has a Unique ID.
- Everything below the row 026 is part of that customer's Unique ID.
- The ReceivedDttm is applicable for all customers in that dataflow file.
- There may be multiple Meter Registers per Customer.
- The Unique ID, MSN, ReadingType and ValidationStatus are the 
  same for all registers for a customer

In [1]:
import pandas as pd
import numpy as np

In [2]:
""" Fucntion to split up customers individually from dataflow file.
    Creates a list of individual dataframes for each customer.
"""

def data_split(dataframe):
    lst= []
    start = 0
    for index, row in dataframe.iterrows():
        df1 = pd.DataFrame()
        if int(row[0]) == 26 and index != 0:
            end = index
            df1 = dataframe[start:end]
            start = index
            lst.append(df1)
        if index == len(dataframe) - 1:
            end = index
            df1 = dataframe[start:end+1]  
            lst.append(df1)  
    return lst

In [3]:
""" Function to align data from dataflow into correct columns to match the expected output layout.
    Creates dictionaries for the expected output column and the data from the dataflow.
    Also creates list of dictionaires for customers that have more than one meter register.
"""

def data_align(dataframe,RDttm):
    dct = {}
    dct['ReceivedDttm'] = RDttm
    lst2 = []
    for index, row in dataframe.iterrows():
        if int(row[0]) == 26:
            dct['MPAN'] = (row[1])
            dct['ValidationStatus'] = (row[2])
        if int(row[0]) == 28:
            dct['MSN'] = (row[1])
            dct['ReadingType'] = (row[2])        
        if int(row[0]) == 30 and row[1] == '1':
            dct['MeterRegister'] = (row[1])
            dct['ReadDttm'] = (row[2])
            dct['Read'] = (row[3]) 
        if int(row[0]) == 30 and int(row[1]) > 1:
            dct2 = {}
            dct2['MeterRegister'] = (row[1])
            dct2['ReadDttm'] = (row[2])
            dct2['Read'] = (row[3]) 
            lst2.append(dct2)
    return dct, lst2

In [4]:
""" Function to appened data to a dataframe.
    Uses the dictionary and list returned by the data_align() fucntion.
    Set fillna method to forward fill to carry over data when a customer has multiple meter registers
"""

def data_append(dct, lst, output):
    output = output.append(dct, ignore_index=True)
    if len(lst) > 0:    
        for dct in lst:
            output = output.append(dct, ignore_index=True)
        output = output.fillna(method='ffill')
    return output

In [5]:
""" Function to take dataflow file name and output expected csv.
    Datafile and outputfile parameters are to be taken as a string.
    The ReceivedDttm is also extracted from the original dataframe and is used in the data_align() function.
"""

def transform_dataflow(datafile, outputfile):
    df1 = pd.read_table(datafile, sep = '|')
    customers = df1[:-1]
    RDttm = list(df1)[5]
    output = pd.DataFrame(columns = ['ReceivedDttm', 'MPAN', 'MSN', 'ReadingType', 'MeterRegister', 'ReadDttm', 'Read', 'ValidationStatus'])
    for df in data_split(customers):
        dct, lst = data_align(df,RDttm)
        output = output.append(data_append(dct, lst, output))
        output = output.drop_duplicates()  
    output['ReceivedDttm'] = pd.to_datetime(output['ReceivedDttm'])
    output['ReadDttm'] = pd.to_datetime(output['ReadDttm'])
    output.to_csv(outputfile, index = False)
    return output

In [12]:
# original dataflow
dataflow = pd.read_table('Dataflow.D10', sep = '|')
dataflow

,Header,XZ1,DataflowD10,X,ABCD,20190125124600
0,026,14813121,U,NaN,NaN,NaN
1,028,SIEMENSx9817898312,A,NaN,NaN,NaN
2,030,1,20190124031257,613.4,NaN,NaN
3,026,1191231,V,NaN,NaN,NaN
4,028,SIEMENSx8909845753,C,NaN,NaN,NaN
5,030,1,20190125093113,39212.0,NaN,NaN
6,030,2,20190125093113,16540.0,NaN,NaN
7,026,16541,V,NaN,NaN,NaN
8,028,PANASONICx6192349432,C,NaN,NaN,NaN
9,030,1,20190127113852,1611.0,NaN,NaN


In [13]:
# transforming dataflow 
transform_data_flow = transform_dataflow('Dataflow.D10', 'ExpectedOut.csv')
transform_data_flow

,ReceivedDttm,MPAN,MSN,ReadingType,MeterRegister,ReadDttm,Read,ValidationStatus
0,2019-01-25 12:46:00,14813121,SIEMENSx9817898312,A,1,2019-01-24 03:12:57,613.4,U
1,2019-01-25 12:46:00,1191231,SIEMENSx8909845753,C,1,2019-01-25 09:31:13,39212.0,V
2,2019-01-25 12:46:00,1191231,SIEMENSx8909845753,C,2,2019-01-25 09:31:13,16540.0,V
3,2019-01-25 12:46:00,16541,PANASONICx6192349432,C,1,2019-01-27 11:38:52,1611.0,V
4,2019-01-25 12:46:00,16541,PANASONICx6192349432,C,2,2019-01-27 11:38:52,91300.0,V
5,2019-01-25 12:46:00,16541,PANASONICx6192349432,C,3,2019-01-27 11:38:52,0.0,V


In [16]:
# the desired output 
ExpectedOutput = pd.read_csv('ExpectedOut.csv')
ExpectedOutput

,ReceivedDttm,MPAN,MSN,ReadingType,MeterRegister,ReadDttm,Read,ValidationStatus
0,2019-01-25 12:46:00,14813121,SIEMENSx9817898312,A,1,2019-01-24 03:12:57,613.4,U
1,2019-01-25 12:46:00,1191231,SIEMENSx8909845753,C,1,2019-01-25 09:31:13,39212.0,V
2,2019-01-25 12:46:00,1191231,SIEMENSx8909845753,C,2,2019-01-25 09:31:13,16540.0,V
3,2019-01-25 12:46:00,16541,PANASONICx6192349432,C,1,2019-01-27 11:38:52,1611.0,V
4,2019-01-25 12:46:00,16541,PANASONICx6192349432,C,2,2019-01-27 11:38:52,91300.0,V
5,2019-01-25 12:46:00,16541,PANASONICx6192349432,C,3,2019-01-27 11:38:52,0.0,V


As you can see, the transform_dataflow() function transformed the original layout of the data into the desired, more readable layout.